In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import functions that read from  INTERIM format and normalize the data
sys.path.append(os.path.abspath('../../src/data'))
from extract_for_model import extract_time_series
from extract_for_model import scale_time_series_single

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse,aic

from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
# Import required paths to input files
# Change the file to import if needed
from data_links import soft_pub_IS as input_IS
from data_links import soft_pub_BS as input_BS

In [3]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=5
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df      

In [4]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name\t::\tTest Stat > C(95%)\t=>\t Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [5]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
        stationary = True
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")
        stationary = False
    return(stationary)

In [6]:
def adfuller_test_silent(series, signif=0.10, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    if p_value <= signif:
        stationary = True
    else:
        stationary = False
    return(stationary)

In [7]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """
    Revert back the differencing to get the forecast to original scale.
    """
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [8]:
def make_stationary(df):
    """
    Function returns:
    a list with stationary data that required no differencing
    a list with stationary data that was differenced one
    a list with all other data differenced twice
    a dictioanry of differencede data
    """
    stationary_0d = []
    non_stationary_0d = []
    dict_train_diff = {}
    
    for company in list(df):
        df_train = df[company]
        stat_bool = []
        
        for name, column in df_train.iteritems():
            stat_bool.append( adfuller_test_silent(column, name=column.name))
        
        if sum(stat_bool) == len(stat_bool):
            stationary_0d.append(company)
            dict_train_diff[company] = df_train
        else:
            non_stationary_0d.append(company)
    print("Stationary data after     0 differentiations %d" %(len(stationary_0d)))
    print("Non-stationary data after 0 differentiations %d" %(len(non_stationary_0d)))     
    print("====")

    stationary_1d = []
    non_stationary_1d = []    
    for company in non_stationary_0d:
        dict_train_diff[company] = df[company].diff().dropna()
        df_train = dict_train_diff[company]
    
        stat_bool = []
        for name, column in df_train.iteritems():
            stat_bool.append( adfuller_test_silent(column, name=column.name))
        
        if sum(stat_bool) == len(stat_bool):
            stationary_1d.append(company)
            
        else:
            non_stationary_1d.append(company)    
    print("Stationary data after     1 differentiation  %d" %(len(stationary_1d)))
    print("Non-stationary data after 1 differentiation  %d" %(len(non_stationary_1d)))
    print("====")

    stationary_2d = []
    non_stationary_2d = []
    for company in non_stationary_1d:
        dict_train_diff[company] = dict_train_diff[company].diff().dropna()
        df_train = dict_train_diff[company]
    
        stat_bool = []
        for name, column in df_train.iteritems():
            stat_bool.append( adfuller_test_silent(column, name=column.name))
        
        if sum(stat_bool) == len(stat_bool):
            stationary_2d.append(company)
        else:
            non_stationary_2d.append(company)  
    print("Stationary data after     2 differentiations %d" %(len(stationary_2d)+len(stationary_1d)))
    print("Non-stationary data after 2 differentiations %d" %(len(non_stationary_2d)))
    print("====")
    stationary_2d.extend(non_stationary_2d)
    return stationary_0d, stationary_1d, stationary_2d, non_stationary_2d, dict_train_diff

In [9]:
def create_dict_by_company(df):  
    """ 
    Function breaks down the dataframe into dictionaries of dataframes, where
    the dictionary key is the company id.
    
    """
    dic = {}
    
    for company_ID, data in df.groupby('company'):
        data = data.reset_index()
        data = data.interpolate()
        data = data.dropna()
        data['date'] = pd.to_datetime(data['date'])
        data = data.set_index('date').drop('company',axis=1)
        
        dic[company_ID] = data
    
    return dic        

In [10]:
def fit_VAR(dict_train_diff,stat_0d,stat_1d,stat_2d):
    """
    Fits VAR models to all paris of companies and data.
    Retrunts a dictionary of forecasts.
    """
    forecast_df = {}
    for company,train_df in dict_train_diff.items():
        model = VAR(train_df)
        model_fitted = model.fit(lag)
    
        forecast_input = train_df.values[-lag:]
        fc = model_fitted.forecast(y=forecast_input,steps=nobs)
        if company in stat_0d:
            forecast_df[company] = pd.DataFrame(fc,index=df.index[-nobs:],columns=df.columns)
        elif company in stat_1d:
            forecast_df[company] = pd.DataFrame(fc,index=df.index[-nobs:],columns=df.columns+'_1d')
        else:
            forecast_df[company] = pd.DataFrame(fc,index=df.index[-nobs:],columns=df.columns+'_2d')
    return forecast_df

In [11]:
def mean_absolute_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_arrays(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [12]:
# *******************************************************************
# Load selected feautures from INCOME STATEMENTS and BALANCE SHEETS
# Combine the data from these two sourcesinto one dataframe
# *******************************************************************

# Read from INCOME STATEMENT
source = input_IS
features = ['IQ_EBIT','IQ_TOTAL_OTHER_OPER','IQ_EARNING_CO','IQ_TOTAL_REV']

IS_df = extract_time_series(input_path = source,\
                              features = features,\
                                 steps = 0,\
                              year_min = 2010,\
                              no_shift = True,\
                               no_test = True,\
                             quarterly = True,\
                                  date = True)

# Read from BALANCE SHEETS
features =  ['IQ_AR','IQ_RE','IQ_TOTAL_ASSETS','IQ_TOTAL_CL']
source = input_BS
BS_df = extract_time_series(input_path = source,\
                              features = features,\
                                 steps = 0,\
                              year_min = 2010,\
                              no_shift = True,\
                               no_test = True,\
                             quarterly = True,\
                                  date = True)

# Drop unwatned features. Save as x to avoid output to terminal
x = IS_df.pop('year')
x = IS_df.pop('quarter')
#
x = BS_df.pop('year')
x = BS_df.pop('quarter')

# Merge dataframe
data_set = IS_df.set_index(['company','date']).join(BS_df.set_index(['company','date'])).interpolate()

====  extract_time_series metric ====
Size of RAW data: (16407, 8)
Size of RESHAPED data without NA: (15922, 8)
No of companies in RESHAPED data: 528
No of companies with more than 1 datapoint 528
====  extract_time_series metric ====
Size of RAW data: (16029, 8)
Size of RESHAPED data without NA: (13899, 8)
No of companies in RESHAPED data: 505
No of companies with more than 1 datapoint 501


In [13]:
data_set = data_set.groupby('company')

In [14]:
# Define number observations and lag size in the model
#
nobs = 1
lag  = 5  

In [15]:
# Filter out the companies that come with not long enough time_series
# The minimum length of time-series muste be = n_obs + lag + 1
#
data_filtered = []
comp_too_short = []

for company_ID, data in data_set:
    if data.count()[0] >= (nobs+lag+5):
        data_filtered.append(data)
    else:
        comp_too_short.append(company_ID)

data_filtered = pd.concat(data_filtered)  

In [16]:
# Divide into the test and train set
test_df  = []
train_df = []

for company_ID,df in data_filtered.groupby('company'):
    df_train, df_test = df[0:-nobs], df[-nobs:]
    test_df.append(df_test)
    train_df.append(df_train)
    
test_df = pd.concat(test_df)
train_df = pd.concat(train_df)

In [17]:
# Normalize using standard scaler
#
features_to_norm = train_df.columns[:]

norm_train_df = train_df.copy()
norm_test_df = test_df.copy()

dict_scalers = {}
for feature in features_to_norm:
    norm_train_df, norm_test_df, param = scale_time_series_single(df_train = norm_train_df,\
                                                                  df_test  = norm_test_df,
                                                                 scalemode = 'standard',
                                                                   feature = [feature])
    dict_scalers.update(param)

In [18]:
# Create dictionaries of companies and dataframes
#
dict_train = create_dict_by_company(train_df)
dict_test  = create_dict_by_company(test_df)
#
dict_train_norm = create_dict_by_company(norm_train_df)
dict_test_norm  = create_dict_by_company(norm_test_df)

In [19]:
stat_0d, stat_1d, stat_2d,nonstat, dict_train_diff = make_stationary(dict_train)

Stationary data after     0 differentiations 0
Non-stationary data after 0 differentiations 437
====


/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:867: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


Stationary data after     1 differentiation  48
Non-stationary data after 1 differentiation  389
====
Stationary data after     2 differentiations 148
Non-stationary data after 2 differentiations 289
====


In [20]:
normstat_0d, normstat_1d, normstat_2d,normnonstat, dict_train_norm_diff = make_stationary(dict_train_norm)

Stationary data after     0 differentiations 0
Non-stationary data after 0 differentiations 437
====
Stationary data after     1 differentiation  48
Non-stationary data after 1 differentiation  389
====
Stationary data after     2 differentiations 147
Non-stationary data after 2 differentiations 290
====


In [21]:
# Fit VAR models to data and make predcitions
#
#
dict_fore = fit_VAR(dict_train_diff,stat_0d,stat_1d,stat_2d)
dict_fore_norm = fit_VAR(dict_train_norm_diff,normstat_0d,normstat_1d,normstat_2d)

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWar

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequenc

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DE

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DE

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ig

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWar

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DE

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequenc

/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DEC will be used.
  % freq, ValueWarning)
/home/lus_karol/ENTER/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency Q-DE

In [22]:
# Reverse the differencing
#
#
dict_res = {}
for company in stat_1d:
    dict_res[company] = invert_transformation(dict_train[company],\
                                              dict_fore[company],\
                                              second_diff=False)
for company in stat_2d:
    dict_res[company] = invert_transformation(dict_train[company],\
                                              dict_fore[company],\
                                              second_diff=True) 
dict_res_norm = {}
for company in normstat_1d:
    dict_res_norm[company] = invert_transformation(dict_train_norm[company],\
                                              dict_fore_norm[company],\
                                              second_diff=False)
for company in normstat_2d:
    dict_res_norm[company] = invert_transformation(dict_train_norm[company],\
                                              dict_fore_norm[company],\
                                              second_diff=True) 

In [23]:
#  De-normalize data
#
#
dict_res_denorm = {}
dict_test_denorm = {}

featrues_to_denorm = ['IQ_TOTAL_REV','IQ_EBIT']

for company, res_data in dict_res_norm.items():
    test_data = dict_test_norm[company]
    for f in featrues_to_denorm:
        res_data[f] = dict_scalers[f].inverse_transform(res_data[f])
        test_data[f]= dict_scalers[f].inverse_transform(test_data[f])

    dict_res_denorm[company]= res_data
    dict_test_denorm[company] = test_data

In [24]:
# Compare with the OLS
#
#
res_OLS =pd.read_csv('model_compare/res_OLS_public_health.csv',index_col='company')

for i, row in res_OLS.iterrows():
    if i not in normnonstat:
        if i in dict_res_denorm:
            res_OLS.at[i,'IQ_TOTAL_REV_VAR_targ']=dict_test[i]['IQ_TOTAL_REV'].values
            res_OLS.at[i,'IQ_TOTAL_REV_VAR_norm']=dict_res_denorm[i]['IQ_TOTAL_REV'].values
            res_OLS.at[i,'IQ_TOTAL_REV_VAR']=dict_res[i]['IQ_TOTAL_REV'].values   
res_OLS.drop('date',axis=1)
res_OLS = res_OLS.dropna()


In [25]:
OLS_mse = mean_squared_error(res_OLS.IQ_TOTAL_REV_target,res_OLS.IQ_TOTAL_REV_OLS)
VAR_mse = mean_squared_error(res_OLS.IQ_TOTAL_REV_VAR_targ,res_OLS.IQ_TOTAL_REV_VAR_norm)

OLS_mae = mean_absolute_error(res_OLS.IQ_TOTAL_REV_target,res_OLS.IQ_TOTAL_REV_OLS)
VAR_mae = mean_absolute_error(res_OLS.IQ_TOTAL_REV_VAR_targ,res_OLS.IQ_TOTAL_REV_VAR_norm)

res_OLS['OLS_err'] = res_OLS.IQ_TOTAL_REV_target - res_OLS.IQ_TOTAL_REV_OLS
res_OLS['OLS_err_per'] = (res_OLS.IQ_TOTAL_REV_target - res_OLS.IQ_TOTAL_REV_OLS)/ res_OLS.IQ_TOTAL_REV_target* 100
res_OLS['VAR_err'] = res_OLS.IQ_TOTAL_REV_VAR_targ -res_OLS.IQ_TOTAL_REV_VAR_norm
res_OLS['VAR_err_per'] = (res_OLS.IQ_TOTAL_REV_VAR_targ- res_OLS.IQ_TOTAL_REV_VAR_norm) /res_OLS.IQ_TOTAL_REV_VAR_targ *100

AttributeError: 'DataFrame' object has no attribute 'IQ_TOTAL_REV_VAR_targ'

In [ ]:
print("     OLS \t VAR")
print("MSE: %.2f \t %.2f" %(OLS_mse,VAR_mse))
print("MAE: %.2f \t %.2f" %(OLS_mae,VAR_mae))

In [ ]:
data = [res_OLS['OLS_err'],res_OLS['VAR_err']]
fig, ax = plt.subplots()
ax.set_title('OLS versus VAR errors')
ax.boxplot(data, showfliers=False)

plt.show()

In [ ]:
data = [res_OLS['OLS_err_per'],res_OLS['VAR_err_per']]
fig, ax = plt.subplots()
ax.set_title('Multiple Samples with Different sizes')
ax.boxplot(data, showfliers=False)

plt.show()